# AI Agents vs Agentic AI: A Practical Comparison

This tutorial demonstrates the key differences between **AI Agents** and **Agentic AI** through a practical example.

## Key Concepts:

- **AI Agent**: You (the developer) orchestrate the workflow, deciding which tools to use and when
- **Agentic AI**: The LLM autonomously decides which tools to use and orchestrates the workflow

We'll compare both approaches using a weather comparison task to illustrate the differences in control, flexibility, and implementation complexity.

## Setup: Installing Dependencies

First, we need to install the OpenAI Python library to interact with GPT models. This will enable us to make API calls for both AI Agent and Agentic AI implementations.

In [30]:
pip install python-dotenv openai


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Configuration: Imports and API Setup

Here we import the necessary libraries and configure the OpenAI API key. 

**Important:** Replace `"sk-..."` with your actual OpenAI API key, or set the `OPENAI_API_KEY` environment variable.

We're using GPT-4 as our model for this demonstration, but you can substitute with `gpt-4o` or other available models.

In [31]:
from openai import OpenAI
import os
from typing import List, Dict

# Initialize OpenAI client
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY") or "sk-..."  # replace with your key
)
MODEL = "gpt-4"  # or gpt-4o if available

## Mock Tools: Weather and Flight Functions

These functions simulate external APIs that both approaches can use:

### Weather Tool
- Returns mock weather data for Austin and San Francisco
- In production, this would call a real weather API like OpenWeatherMap

### Flight Tool (New!)
- Simulates a flight search API
- Returns 3 flight options with random but realistic data:
  - Different airlines (Southwest, American, Delta, United, JetBlue)
  - Prices ranging from $200-$800
  - Flight durations from 2-8 hours
  - Departure times throughout the day

The **Agentic AI** can now autonomously decide whether to use weather data, flight data, or both depending on the user's request!

In [32]:
# Simulated tools (could be any external functions)
def get_weather(city: str) -> str:
    fake_data = {
        "Austin": "Sunny, 95°F",
        "San Francisco": "Foggy, 65°F"
    }
    return fake_data.get(city, f"No data available for {city}")

def get_flights(destination: str) -> str:
    import random
    
    # Simulate flight search with 3 options
    airlines = ["Southwest", "American", "Delta", "United", "JetBlue"]
    
    flights = []
    for i in range(3):
        airline = random.choice(airlines)
        price = random.randint(200, 800)
        duration = f"{random.randint(2, 8)}h {random.randint(0, 59)}m"
        departure = f"{random.randint(6, 22):02d}:{random.randint(0, 59):02d}"
        
        flights.append(f"Flight {i+1}: {airline} - ${price}, {duration}, departs {departure}")
    
    return f"Flights to {destination}:\n" + "\n".join(flights)

## Approach 1: AI Agent (Developer-Orchestrated)

In this approach, **you (the developer) control the workflow**:

1. **You decide** which tools to call and when
2. **You parse** the user input to determine the logic
3. **You collect** the data from tools manually
4. **You format** the prompt for the LLM

**Key characteristics:**
- Predictable execution flow
- Explicit control over tool usage
- Simple implementation
- Limited flexibility for complex scenarios

Notice how the developer hardcodes the logic: "if 'compare' in user_input" determines when to fetch weather data for both cities.

In [33]:
def run_ai_agent(user_input: str):
    print("🔧 Running AI Agent...")

    # You decide what tools to run
    if "compare" in user_input.lower():
        austin = get_weather("Austin")
        sf = get_weather("San Francisco")
        prompt = f"Compare the following:\nAustin weather: {austin}\nSan Francisco weather: {sf}"

        response = client.chat.completions.create(
            model=MODEL,
            messages=[{"role": "user", "content": prompt}]
        )

        print("✅ AI Agent Response:\n", response.choices[0].message.content)

## Approach 2: Agentic AI (LLM-Orchestrated)

In this approach, **the LLM controls the workflow**:

1. **The LLM decides** which tools to call and when
2. **The LLM analyzes** the user input autonomously
3. **The LLM determines** the sequence of tool calls needed
4. **The LLM iterates** until it has enough information to respond

**Key characteristics:**
- Flexible and adaptive execution
- LLM-driven decision making
- Handles complex multi-step scenarios
- More sophisticated but potentially unpredictable

The magic happens in the `while True:` loop where the LLM can make multiple tool calls and reason about the results before providing a final answer. Notice how we define tools using OpenAI's function calling format, and the LLM autonomously decides when and how to use them.

In [ ]:
def run_agentic_ai(user_input: str):
    print(f"\n\n\n🧠 Running Agentic AI...")
    messages = [{"role": "user", "content": user_input}]
    
    while True:
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=[
                {
                    "type": "function",
                    "function": {
                        "name": "get_weather",
                        "description": "Get current weather in a city",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "city": {"type": "string"}
                            },
                            "required": ["city"]
                        }
                    }
                },
                {
                    "type": "function",
                    "function": {
                        "name": "get_flights",
                        "description": "Search for flights to a destination city with prices and schedules",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "destination": {"type": "string", "description": "The destination city to search flights for"}
                            },
                            "required": ["destination"]
                        }
                    }
                }
            ],
            tool_choice="auto"
        )

        msg = response.choices[0].message

        if msg.tool_calls:
            # Add assistant message with tool calls
            messages.append({
                "role": "assistant",
                "content": msg.content,
                "tool_calls": msg.tool_calls
            })
            
            for tool_call in msg.tool_calls:
                name = tool_call.function.name
                args = tool_call.function.arguments
                import json
                parsed_args = json.loads(args)
                
                # Call the appropriate function based on the tool name
                if name == "get_weather":
                    result = get_weather(**parsed_args)
                elif name == "get_flights":
                    result = get_flights(**parsed_args)
                else:
                    result = f"Unknown function: {name}"

                # Add tool result to conversation
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": result
                })
        else:
            # Final answer
            messages.append({
                "role": "assistant",
                "content": msg.content
            })
            print("✅ Agentic AI Response:\n", msg.content)
            break

## Demo: Comparing Both Approaches

Let's run both implementations with the same user question: "Compare the weather in Austin and San Francisco"

**What to expect:**

1. **AI Agent**: Will detect "compare" in the input, fetch weather for both cities, and ask GPT-4 to compare them
2. **Agentic AI**: Will analyze the request, decide to call the weather tool twice (once for each city), then provide a comparison

**Key Differences You'll Observe:**
- **Speed**: AI Agent is faster (fewer API calls)
- **Flexibility**: Agentic AI can handle variations in phrasing better
- **Control**: AI Agent gives you explicit control over the workflow
- **Adaptability**: Agentic AI can handle unexpected scenarios more gracefully

Run the cell below to see both approaches in action!

In [ ]:
user_question = "Compare the weather in Austin and San Francisco"

# AI Agent (you orchestrate)
run_ai_agent(user_question)

# Agentic AI (model orchestrates)
run_agentic_ai(user_question)

# Test the new flight functionality
flight_question = "I want to fly to Austin. Show me flight options and the weather there."
print(f"\nUser: {flight_question}")

# Only Agentic AI can handle this complex request automatically
run_agentic_ai(flight_question)

🔧 Running AI Agent...
✅ AI Agent Response:
 The weather in Austin is much hotter than in San Francisco with a temperature difference of 30°F. Austin experiences sunny weather, indicating clear skies and intense heat which is typical of a warm or hot climate. On the other hand, San Francisco is experiencing foggy weather, suggesting low visibility and cooler temperatures often associated with coastal or cold climates. The significant difference in weather between these two cities could be due to their geographical location and the time of the year.



🧠 Running Agentic AI...
✅ Agentic AI Response:
 The weather in Austin is currently sunny with a temperature of 95°F. On the other hand, it's foggy in San Francisco with a temperature of 65°F.

🆕 NEW: Try asking about flights!

User: I want to fly to Austin. Show me flight options and the weather there.



🧠 Running Agentic AI...
✅ Agentic AI Response:
 Here are the flight options to Austin:

- American - $269, 3h 9m, departs 20:31
- JetBlu